<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/machine_learning/blob/main/classes/class_march_17/clase_march_17_clasificacion_multietiqueta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Class del 22 de marzo Precision and recall

## Intercambio de precisión/recuperación

Para comprender esta compensación, veamos cómo `SGDClassifier` toma sus decisiones de clasificación.

Para cada instancia, calcula una puntuación basada en una función de decisión.

Si esa puntuación es mayor que un umbral, asigna la instancia a la clase positiva; de lo contrario, lo asigna a la clase negativa.

La siguiente figura muestra algunos dígitos ubicados desde la puntuación más baja a la izquierda hasta la puntuación más alta a la derecha.

<img src = 'https://github.com/marco-canas/taca/blob/main/ref/geron/part_1/chap_3/1_MNIST/figura_3_3_clasificacion_binaria_mnist.jpg?raw=true'>

Supongamos que el umbral de decisión se coloca en la flecha central (entre los dos 5): encontrará 4 verdaderos positivos (5 reales) a la derecha de ese umbral y 1 falso positivo (en realidad, un 6).

Por tanto, con ese umbral, la precisión es del 80% (4 sobre 5).

Pero de los 6 cincos reales, el clasificador solo detecta 4, por lo que la recuperación es del $67 \%$ (4 de 6).

Si subes el umbral (muévelo hacia la flecha de la derecha), el falso positivo (el 6) se convierte en un verdadero negativo, aumentando así la precisión (hasta el 100% en este caso), pero un verdadero positivo se convierte en un falso negativo , disminuyendo el recuerdo hasta un $50\%$.

Por el contrario, bajar el umbral aumenta la recuperación y reduce la precisión.

Scikit-Learn no le permite establecer el umbral directamente, pero le da acceso a los puntajes de decisión que utiliza para hacer predicciones. 

En lugar de llamar al método `predict()` del clasificador, puede llamar a su método `decision_function()`, que devuelve un puntaje para cada instancia, y luego usar cualquier umbral que desee para hacer predicciones basadas en esos puntajes:

In [ ]:
%%time 

import numpy as np 
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False) 

In [ ]:
X,y = mnist['data'], mnist['target'] 

In [ ]:
y = y.astype(np.int64) 

In [ ]:
X_train, X_test, y_train, y_test = X[:60_000], X[60_000:], y[:60_000], y[60_000:]

In [ ]:
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_clf = SGDClassifier(max_iter = 1000, tol = 1e-3, random_state = 42) 

In [ ]:
%%time

sgd_clf.fit(X_train, y_train_5) 

In [ ]:
y_scores = sgd_clf.decision_function([X[0]])
y_scores

# Función de decisión personalizada

In [ ]:
def funcion_decision(imagen, threshold):
    y_score = sgd_clf.decision_function([imagen])
    return y_score>threshold

In [ ]:
threshold = 0
y_some_digit_pred = (y_scores > threshold)
y_some_digit_pred

El `SGDClassifier` usa un umbral igual a 0, por lo que el código anterior devuelve el mismo resultado que el método `predict()` (es decir, True).

Subamos el umbral:

In [ ]:
threshold = 8000
y_some_digit_pred = (y_scores > threshold)
y_some_digit_pred

Esto confirma que elevar el umbral disminuye el recuerdo.

La imagen en realidad representa un 5, y el clasificador lo detecta cuando el umbral es 0, pero lo pierde cuando el umbral aumenta a 8000.

¿Cómo se decide qué umbral usar?

Primero, use la función `cross_val_predict()` para obtener los puntajes de todas las instancias en el conjunto de entrenamiento, pero esta vez especifique que desea obtener puntajes de decisión en lugar de predicciones:

In [ ]:
from sklearn.model_selection import cross_val_predict 

In [ ]:
%%time 

y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

Con estas puntuaciones, utilice la función `precision_recall_curve()` para calcular la precisión y la recuperación de todos los umbrales posibles:

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

Finalmente, use Matplotlib para trazar la precisión y recordar como funciones del valor de umbral (Figura 3-4):

In [ ]:
import matplotlib.pyplot as plt 

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    [...] # highlight the threshold and add the legend, axis label, and grid
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

Figura 3_4 
<img src = ''>

## NOTE

Quizás se pregunte por qué la curva de precisión es más irregular que la curva de recuperación de la figura 3-4.

El motivo es que, en ocasiones, la precisión puede disminuir cuando aumenta el umbral (aunque, en general, aumentará).

Para comprender por qué, vuelva a mirar la Figura 3-3 y observe lo que sucede cuando comienza desde el umbral central y lo mueve solo un dígito a la derecha: la precisión va de 4/5 (80 %) a 3/4 (75 %). ).

Por otro lado, la recuperación solo puede disminuir cuando se aumenta el umbral, lo que explica por qué su curva parece suave.

Otra forma de seleccionar una buena compensación de precisión/recuperación es graficar la precisión directamente contra la recuperación, como se muestra en la Figura 3-5 (se resalta el mismo umbral que antes).

Puede ver que la precisión realmente comienza a caer bruscamente alrededor del $80\%$ de recuerdo.

You will probably want to select a precision/recall trade-off just before that drop—for example, at around 60% recall. 

But of course, the choice depends on your project.

Suppose you decide to aim for 90% precision. 

You look up the first plot and find that you need to use a threshold of about 8,000. 

To be more precise you can search for the lowest threshold that gives you at least 90% precision (np.argmax() will give you the first index of the maximum value, which in this case means the first True value):

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)] 

In [ ]:
threshold_90_precision

To make predictions (on the training set for now), instead of calling the classifier’s `predict()` method, you can run this code:

In [ ]:
y_train_pred_90 = (y_scores >= threshold_90_precision)

Let’s check these predictions’ precision and recall:

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(y_train_5, y_train_pred_90)

In [ ]:
recall_score(y_train_5, y_train_pred_90)

Great, you have a 90% precision classifier! 

Como puede ver, es bastante fácil crear un clasificador con prácticamente cualquier precisión que desee: simplemente establezca un umbral suficientemente alto, y está listo.

But wait, not so fast. 

A high-precision classifier is not very useful if its recall is too low!

### Sugerencia 

If someone says, “Let’s reach 99% precision,” you should ask, “At what recall?”

## The ROC Curve

The receiver operating characteristic (ROC) curve is another common tool used with binary classifiers. 

It is very similar to the precision/recall curve, but instead of plotting precision versus recall, the ROC curve plots the true positive rate (another name for recall) against the false positive rate (FPR).

The FPR is the ratio of negative instances that are incorrectly classified as positive. 

It is equal to 1 – the true negative rate (TNR), which is the ratio of negative instances that are correctly classified as negative. 

The TNR is also called specificity. Hence, the ROC curve plots sensitivity (recall) versus 1 –
specificity.

To plot the ROC curve, you first use the `roc_curve()` function to compute the TPR and FPR for various threshold values:

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

Then you can plot the FPR against the TPR using Matplotlib. 

This code produces the plot in Figure 3-6:

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
    [...] # Add axis labels and grid
plot_roc_curve(fpr, tpr)
plt.show()

Once again there is a trade-off: the higher the recall (TPR), the more false positives (FPR) the classifier produces. 

The dotted line represents the ROC curve of a purely random classifier; a good classifier stays as far away from that line as possible (toward the top-left corner).

One way to compare classifiers is to measure the area under the curve (AUC).  

A perfect classifier will have a ROC AUC equal to 1, whereas a purely random classifier will have a ROC AUC equal to 0.5. Scikit-Learn provides a function to compute the ROC AUC:

## Sugerencia 


Dado que la curva ROC es tan similar a la curva de precisión/recuerdo (PR), puede preguntarse cómo decidir cuál usar.

As a rule of thumb, you should prefer the PR curve whenever the positive class is rare or when you care more about the false positives than the false negatives. 

Otherwise, use the ROC curve. For example, looking at the previous ROC curve (and the ROC AUC score), you may think that the classifier is really good.

But this is mostly because there are few positives (5s) compared to the negatives (non - 5s). 

In contrast, the PR curve makes it clear that the classifier has room for improvement
(the curve could be closer to the top-left corner).

Let’s now train a RandomForestClassifier and compare its ROC curve and ROC AUC score to those of the SGDClassifier. 

First, you need to get scores for each instance in the training set. 

But due to the way it works (see Chapter 7), the `RandomForestClassifier` class does not have a
`decision_function()` method. Instead, it has a `predict_proba()` method. 

Scikit-Learn classifiers generally have one or the other, or both.

The `predict_proba()` method returns an array containing a row per instance and a column per class, each containing the probability that the given instance belongs to the given class (e.g., 70% chance that the image represents a 5):

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
method="predict_proba")

The roc_curve() function expects labels and scores, but instead of scores you can give it class probabilities. 

Let’s use the positive class’s probability as the score:

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

Now you are ready to plot the ROC curve. 

It is useful to plot the first ROC curve as well to see how they compare (Figure 3-7):

In [ ]:
plt.plot(fpr, tpr, "b:", label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right")
plt.show()

As you can see in Figure 3-7, the RandomForestClassifier’s ROC curve looks much better than the `SGDClassifier’s`: it comes much closer to the top-left corner. 

As a result, its ROC AUC score is also significantly better:

In [ ]:
from sklearn.metrics import roc_auc_score 

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)

Try measuring the precision and recall scores: you should find $99.0\%$ precision and $86.6\%$ recall. 

Not too bad!

You now know how to train binary classifiers, choose the appropriate metric for your task, evaluate your classifiers using cross-validation, select the precision/recall trade-off that fits your needs, and use ROC curves and ROC AUC scores to compare various models. 

Now let’s try to detect more than just the 5s.